# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/12_vision_avancee/12_exercices.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print('📦 Installation des packages...')
    !pip install -q torch torchvision
    !pip install -q ultralytics  # YOLOv8
    !pip install -q segmentation-models-pytorch
    !pip install -q albumentations
    !pip install -q numpy pandas matplotlib seaborn Pillow opencv-python
    print('✅ Installation terminée !')
else:
    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 12 - Exercices : Vision Avancée

Ce notebook contient des exercices pratiques pour consolider les concepts du Chapitre 12.

**Instructions** :
- Complétez les cellules marquées `# VOTRE CODE ICI`
- Les solutions sont disponibles dans `12_exercices_solutions.ipynb`
- N'hésitez pas à consulter la documentation (Ultralytics, PyTorch, segmentation-models-pytorch)

**Avertissement** : Ces exercices nécessitent des ressources GPU pour un entraînement efficace. Sur CPU, limitez les epochs et la taille des modèles.

---

## Setup Initial

In [ ]:
# Imports nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from ultralytics import YOLO
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

import warnings
warnings.filterwarnings('ignore')

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
np.random.seed(42)
torch.manual_seed(42)

# Détection GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device : {device}")

print("✓ Bibliothèques importées")

---

## Exercice 1 : IoU et NMS (Fondamentaux)

**Objectif** : Implémenter les métriques de base pour l'object detection.

### 1.1 Intersection over Union (IoU)

In [ ]:
# Implémentez IoU
# VOTRE CODE ICI

def compute_iou(box1, box2):
    """
    Calcule l'IoU entre deux bounding boxes.
    Args:
        box1, box2: [x1, y1, x2, y2] format (coin haut-gauche, coin bas-droit)
    Returns:
        IoU (float entre 0 et 1)
    """
    # TODO: Calculez l'aire d'intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = None  # max(0, x2 - x1) * max(0, y2 - y1)
    
    # TODO: Calculez les aires de box1 et box2
    area1 = None
    area2 = None
    
    # TODO: Calculez l'union
    union = None
    
    # TODO: Calculez l'IoU
    iou = None
    
    return iou

# Tests
box_a = [10, 10, 50, 50]  # Carré 40x40
box_b = [30, 30, 70, 70]  # Carré 40x40, chevauchement partiel
box_c = [10, 10, 50, 50]  # Identique à box_a
box_d = [100, 100, 140, 140]  # Pas de chevauchement

print(f"IoU(box_a, box_b) = {compute_iou(box_a, box_b):.4f}")
print(f"IoU(box_a, box_c) = {compute_iou(box_a, box_c):.4f} (attendu : 1.0)")
print(f"IoU(box_a, box_d) = {compute_iou(box_a, box_d):.4f} (attendu : 0.0)")

### 1.2 Non-Maximum Suppression (NMS)

In [ ]:
# Implémentez NMS
# VOTRE CODE ICI

def non_max_suppression(boxes, scores, iou_threshold=0.5):
    """
    Applique NMS pour éliminer les détections redondantes.
    Args:
        boxes: Liste de [x1, y1, x2, y2]
        scores: Liste de scores de confiance
        iou_threshold: Seuil d'IoU pour supprimer les boxes
    Returns:
        Indices des boxes à garder
    """
    # TODO: Triez les boxes par score décroissant
    # TODO: Pour chaque box dans l'ordre :
    #   - Gardez-la si elle n'a pas été supprimée
    #   - Supprimez toutes les boxes avec IoU > iou_threshold
    # TODO: Retournez les indices des boxes conservées
    pass

# Test
boxes_test = [
    [10, 10, 50, 50],
    [15, 15, 55, 55],  # Très similaire à la première
    [100, 100, 140, 140],  # Différente
    [12, 12, 52, 52]  # Très similaire à la première
]
scores_test = [0.9, 0.8, 0.95, 0.7]

kept_indices = non_max_suppression(boxes_test, scores_test, iou_threshold=0.5)
print(f"Boxes conservées : {kept_indices}")
print(f"Attendu : conserver box 0 (score 0.9) et box 2 (score 0.95, différente zone)")

### 1.3 Visualisation

In [ ]:
# Visualisez l'effet du NMS
# VOTRE CODE ICI

def draw_boxes(image, boxes, scores, labels, color=(255, 0, 0), thickness=2):
    """Dessine les bounding boxes sur une image."""
    img_copy = image.copy()
    for i, (box, score, label) in enumerate(zip(boxes, scores, labels)):
        x1, y1, x2, y2 = box
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), color, thickness)
        text = f"{label} {score:.2f}"
        cv2.putText(img_copy, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return img_copy

# TODO: Créez une image blanche 200x200
# TODO: Dessinez les boxes avant et après NMS côte à côte

---

## Exercice 2 : YOLOv8 pour Object Detection

**Objectif** : Utiliser YOLOv8 pour détecter des objets dans des images.

### 2.1 Chargement du Modèle Pré-entraîné

In [ ]:
# Chargez YOLOv8 nano (le plus léger)
# VOTRE CODE ICI

model_yolo = None  # YOLO('yolov8n.pt')  # n = nano, s = small, m = medium, l = large

print("✓ Modèle YOLOv8 chargé")
print(f"Classes disponibles : {len(model_yolo.names)}")
print(f"Exemples : {list(model_yolo.names.values())[:10]}")

### 2.2 Détection sur une Image

In [ ]:
# Téléchargez ou utilisez une image test
# VOTRE CODE ICI

# Option 1 : Téléchargez depuis URL
# !wget https://ultralytics.com/images/bus.jpg -O test_image.jpg

# Option 2 : Utilisez une image locale
# img_path = 'test_image.jpg'

# TODO: Effectuez la détection
# results = model_yolo(img_path)

# TODO: Affichez les résultats
# results[0].show()  # Affiche l'image avec les détections

### 2.3 Analyse des Détections

In [ ]:
# Extrayez les informations des détections
# VOTRE CODE ICI

# TODO: Parcourez les détections et extrayez :
# - Bounding boxes
# - Scores de confiance
# - Classes détectées

# for result in results:
#     boxes = result.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
#     scores = result.boxes.conf.cpu().numpy()
#     classes = result.boxes.cls.cpu().numpy()
#     
#     for box, score, cls in zip(boxes, scores, classes):
#         class_name = model_yolo.names[int(cls)]
#         print(f"{class_name} : {score:.3f} | Box : {box}")

### 2.4 Fine-tuning YOLOv8 (Optionnel - Bonus)

Pour cet exercice, vous devez préparer un dataset au format YOLO (images + annotations).

In [ ]:
# Fine-tuning YOLOv8 sur un dataset personnalisé
# VOTRE CODE ICI

# Structure attendue du dataset :
# dataset/
#   train/
#     images/
#     labels/  # Fichiers .txt avec format : class x_center y_center width height (normalisés 0-1)
#   val/
#     images/
#     labels/
#   data.yaml  # Fichier de configuration

# Contenu de data.yaml :
# train: /path/to/dataset/train/images
# val: /path/to/dataset/val/images
# nc: 2  # Nombre de classes
# names: ['class1', 'class2']

# TODO: Entraînez le modèle
# model_yolo.train(data='data.yaml', epochs=10, imgsz=640, batch=16)

---

## Exercice 3 : U-Net pour Segmentation Sémantique

**Objectif** : Implémenter et entraîner un U-Net sur un dataset synthétique.

### 3.1 Génération de Données Synthétiques

In [ ]:
# Générez un dataset synthétique (formes géométriques)
# VOTRE CODE ICI

def generate_synthetic_data(n_samples=1000, img_size=128):
    """
    Génère des images avec des formes géométriques et leurs masques.
    Returns:
        images: (n_samples, img_size, img_size, 3)
        masks: (n_samples, img_size, img_size) avec classes 0=background, 1=circle, 2=rectangle
    """
    images = []
    masks = []
    
    for _ in range(n_samples):
        # TODO: Créez une image blanche
        img = np.ones((img_size, img_size, 3), dtype=np.uint8) * 255
        mask = np.zeros((img_size, img_size), dtype=np.uint8)
        
        # TODO: Ajoutez aléatoirement des cercles et rectangles
        # Utilisez cv2.circle et cv2.rectangle
        # Remplissez le masque avec les labels correspondants
        
        # Exemple : Cercle
        if np.random.rand() > 0.5:
            center = (np.random.randint(20, img_size-20), np.random.randint(20, img_size-20))
            radius = np.random.randint(10, 30)
            color = tuple(np.random.randint(0, 255, 3).tolist())
            cv2.circle(img, center, radius, color, -1)
            cv2.circle(mask, center, radius, 1, -1)  # Label 1 pour cercle
        
        # TODO: Ajoutez un rectangle de la même manière (label 2)
        
        images.append(img)
        masks.append(mask)
    
    return np.array(images), np.array(masks)

# Générez le dataset
X_train, y_train = generate_synthetic_data(n_samples=800, img_size=128)
X_val, y_val = generate_synthetic_data(n_samples=200, img_size=128)

print(f"Train : {X_train.shape}, {y_train.shape}")
print(f"Val : {X_val.shape}, {y_val.shape}")

### 3.2 Visualisation du Dataset

In [ ]:
# Visualisez quelques exemples
# VOTRE CODE ICI

fig, axes = plt.subplots(3, 4, figsize=(12, 9))

for i in range(3):
    idx = np.random.randint(0, len(X_train))
    
    # Image
    axes[i, 0].imshow(X_train[idx])
    axes[i, 0].set_title('Image')
    axes[i, 0].axis('off')
    
    # Mask
    axes[i, 1].imshow(y_train[idx], cmap='tab20')
    axes[i, 1].set_title('Mask')
    axes[i, 1].axis('off')
    
    # TODO: Affichez les canaux séparés pour chaque classe

plt.tight_layout()
plt.show()

### 3.3 Dataset et DataLoader PyTorch

In [ ]:
# Créez un Dataset PyTorch
# VOTRE CODE ICI

class SegmentationDataset(Dataset):
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        
        # TODO: Appliquez les transformations
        if self.transform:
            # Utilisez albumentations pour transformer image ET mask ensemble
            pass
        
        # TODO: Convertissez en tenseurs
        # Image : (C, H, W) normalisée [0, 1]
        # Mask : (H, W) avec labels entiers
        
        return image, mask

# Transformations
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Créez les datasets et dataloaders
train_dataset = SegmentationDataset(X_train, y_train, transform=train_transform)
val_dataset = SegmentationDataset(X_val, y_val, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print(f"✓ Dataloaders créés")

### 3.4 Modèle U-Net

In [ ]:
# Utilisez segmentation-models-pytorch pour créer un U-Net
# VOTRE CODE ICI

# TODO: Créez le modèle
model = smp.Unet(
    encoder_name='resnet18',        # Encoder : resnet18, resnet34, efficientnet-b0, etc.
    encoder_weights='imagenet',     # Poids pré-entraînés sur ImageNet
    in_channels=3,                  # RGB
    classes=3                       # Nombre de classes (0=bg, 1=circle, 2=rectangle)
)

model = model.to(device)

print(f"✓ Modèle U-Net créé")
print(f"Nombre de paramètres : {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

### 3.5 Loss Function et Entraînement

In [ ]:
# Définissez la loss function
# VOTRE CODE ICI

# TODO: Utilisez DiceLoss ou une combinaison Dice + CrossEntropy
loss_fn = smp.losses.DiceLoss(mode='multiclass')

# TODO: Créez l'optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# TODO: Créez le scheduler (optionnel)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)

print("✓ Loss et optimizer créés")

In [ ]:
# Boucle d'entraînement
# VOTRE CODE ICI

def train_epoch(model, loader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    
    for images, masks in loader:
        # TODO: Entraînement standard
        # 1. Move to device
        # 2. Forward pass
        # 3. Compute loss
        # 4. Backward
        # 5. Optimizer step
        pass
    
    return total_loss / len(loader)

def val_epoch(model, loader, loss_fn, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for images, masks in loader:
            # TODO: Validation
            pass
    
    return total_loss / len(loader)

# TODO: Boucle d'entraînement principale
n_epochs = 10
train_losses = []
val_losses = []

for epoch in range(n_epochs):
    # TODO: Entraînez et validez
    pass

print("\n✓ Entraînement terminé")

### 3.6 Visualisation des Prédictions

In [ ]:
# Visualisez les prédictions sur le validation set
# VOTRE CODE ICI

model.eval()

fig, axes = plt.subplots(4, 3, figsize=(12, 16))

with torch.no_grad():
    for i in range(4):
        idx = np.random.randint(0, len(val_dataset))
        image, mask = val_dataset[idx]
        
        # TODO: Prédiction
        # pred = model(image.unsqueeze(0).to(device))
        # pred_mask = torch.argmax(pred, dim=1).cpu().numpy()[0]
        
        # TODO: Affichez image, mask GT, mask prédite

plt.tight_layout()
plt.show()

---

## Exercice 4 : Métriques de Segmentation

**Objectif** : Implémenter les métriques d'évaluation pour la segmentation.

### 4.1 Dice Coefficient

In [ ]:
# Implémentez le Dice Coefficient (F1-score pour segmentation)
# VOTRE CODE ICI

def dice_coefficient(pred, target, smooth=1e-6):
    """
    Calcule le Dice Coefficient.
    Args:
        pred: Prédictions binaires (H, W)
        target: Ground truth binaire (H, W)
        smooth: Constante pour éviter division par zéro
    Returns:
        Dice score (float entre 0 et 1)
    """
    # TODO: Dice = 2 * |A ∩ B| / (|A| + |B|)
    pass

# Test
pred_test = np.array([[1, 1, 0], [1, 0, 0], [0, 0, 0]])
target_test = np.array([[1, 1, 0], [1, 1, 0], [0, 0, 0]])

dice = dice_coefficient(pred_test, target_test)
print(f"Dice Coefficient : {dice:.4f}")

### 4.2 IoU par Classe

In [ ]:
# Calculez l'IoU pour chaque classe
# VOTRE CODE ICI

def compute_iou_multiclass(pred, target, n_classes):
    """
    Calcule l'IoU pour chaque classe.
    Args:
        pred: Prédictions (H, W) avec labels 0, 1, 2, ...
        target: Ground truth (H, W)
        n_classes: Nombre de classes
    Returns:
        Liste d'IoU pour chaque classe
    """
    ious = []
    
    for cls in range(n_classes):
        # TODO: Créez des masques binaires pour la classe cls
        # TODO: Calculez IoU pour cette classe
        pass
    
    return ious

# TODO: Testez sur vos prédictions

---

## Exercice 5 : Questions de Réflexion

**Question 1** : Quelle est la différence entre segmentation sémantique et segmentation d'instance ?

**VOTRE RÉPONSE ICI**

---

**Question 2** : Pourquoi utilise-t-on Dice Loss plutôt que CrossEntropy pour la segmentation ?

**VOTRE RÉPONSE ICI**

---

**Question 3** : À quoi servent les skip connections dans U-Net ?

**VOTRE RÉPONSE ICI**

---

**Question 4** : Comparez YOLOv8 et Faster R-CNN en termes de vitesse et précision.

**VOTRE RÉPONSE ICI**

---

## Conclusion

Félicitations pour avoir complété ces exercices !

**Points clés à retenir** :
- IoU et NMS sont fondamentaux pour l'object detection
- YOLOv8 offre un excellent compromis vitesse/précision pour la détection temps réel
- U-Net est l'architecture de référence pour la segmentation sémantique
- Les skip connections préservent les détails spatiaux
- Dice Loss et IoU sont les métriques standards pour la segmentation

**Prochaines étapes** :
- Consultez les solutions dans `12_exercices_solutions.ipynb`
- Testez YOLOv8 sur vos propres images
- Entraînez U-Net sur des datasets réels (médicaux, satellites)
- Explorez Mask R-CNN pour la segmentation d'instance
- Passez au Chapitre 13 (Systèmes de Recommandation)

---